Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [3]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [4]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [5]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [7]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [8]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
id       49159 non-null int64
label    31962 non-null float64
tweet    49159 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


3. Изменим регистр твитов на нижний с помощью .lower()

4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15. Сохраним результат предобработки в pickle-файл.

In [9]:
!pip install HTMLParser

In [10]:
#import HTMLParser
#parser = HTMLParser()
#combine_df['tweet'] = combine_df['tweet'].apply(lambda x: parser.unescape(x))

In [11]:
def change_text(text):
    text = re.sub(r'@[\w]*',' ', text)
    text = text.lower()
    new_text = ''
    for word in text.split():
        if word in apostrophe_dict.keys():
            word = apostrophe_dict[word]
        if word in short_word_dict.keys():
            word = short_word_dict[word]
        if word in emoticon_dict.keys():
            word = emoticon_dict[word]            
        new_text = new_text + word + ' ' 
    new_text = re.sub(r'[^\w\s]',' ', new_text)
    new_text = re.sub(r'[^a-zA-Z]',' ', new_text)

    new_text = ' '.join([w for w in new_text.split() if len(w)>1])
    return new_text

In [12]:
#change_text_vect = np.vectorize(change_text)
combine_df['tweet'] = combine_df['tweet'].apply(lambda x: change_text(x))

In [13]:
combine_df['tweet'].head(2)

0    when father is dysfunctional and is so selfish...
1    thanks for lyft credit cannot use cause they d...
Name: tweet, dtype: object

In [14]:
import nltk
nltk.download('punkt')
combine_df['tweet_token'] = combine_df['tweet'].apply(lambda x: nltk.tokenize.word_tokenize(x))
combine_df['tweet_token'].head(2)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    [when, father, is, dysfunctional, and, is, so,...
1    [thanks, for, lyft, credit, can, not, use, cau...
Name: tweet_token, dtype: object

In [15]:
#combine_df['tweet_token'] = combine_df['tweet'].apply(lambda x: x.split())
#combine_df['tweet_token'].head(2)

In [16]:
#Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
stop_words = set(stopwords.words("english"))
sentence = "Backgammon is one of the oldest known board games."

words = sentence.split()
without_stop_words = [word for word in words if not word in stop_words]
print(without_stop_words)

['Backgammon', 'one', 'oldest', 'known', 'board', 'games.']


In [20]:
stop_words = set(stopwords.words("english"))

def get_without_stop_words(words, stop_words):
    words = list(words)
    without_stop_words = [word for word in words if not word in stop_words]
    return without_stop_words

In [21]:
combine_df['tweet_token_filtered'] = combine_df['tweet_token'].apply(lambda x: get_without_stop_words(x, stop_words))
combine_df['tweet_token_filtered']

0        [father, dysfunctional, selfish, drags, kids, ...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                            [model, love, take, time, ur]
4                        [factsguide, society, motivation]
5        [huge, fan, fare, big, talking, leave, chaos, ...
6                               [camping, tomorrow, danny]
7        [next, school, year, year, exams, think, schoo...
8        [love, land, allin, cavs, champions, cleveland...
9                                            [welcome, gr]
10       [ireland, consumer, price, index, mom, climbed...
11       [selfish, orlando, standwithorlando, pulseshoo...
12              [get, see, daddy, today, days, gettingfed]
13       [cnn, calls, michigan, middle, school, build, ...
14       [comment, australia, opkillingbay, seashepherd...
15        [ouch, junior, angry, got, junior, yugyoem, omg]
16                   [thankful, paner, thankful, positiv

In [22]:
#Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.
#Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [23]:
!pip install nltk

In [24]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

def get_stemmer(stemmer, words):
    """
    Print the results of stemmind using the passed stemmer, lemmatizer, word and pos (part of speech)
    """
    res = []
    for word in words:
        res.append(stemmer.stem(word))
    return res      
    
def get_lemmatizer(lemmatizer, words, pos):
    """
    Print the results of lemmitization using the passed stemmer, lemmatizer, word and pos (part of speech)
    """
    res = []
    for word in words:
        res.append(lemmatizer.lemmatize(word, pos))
    return res      
    

In [25]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [37]:
def get_lem(tweet, lemmatizer):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tweet]


In [38]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
combine_df['tweet_stemmed'] = combine_df['tweet_token_filtered'].apply(lambda x: get_stemmer(stemmer, x))
combine_df['tweet_lemmatized'] = combine_df['tweet_token_filtered'].apply(lambda x: get_lem(x, lemmatizer))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [39]:
combine_df['tweet_stemmed']

0        [father, dysfunct, selfish, drag, kid, dysfunc...
1        [thank, lyft, credit, use, caus, offer, wheelc...
2                                        [bihday, majesti]
3                            [model, love, take, time, ur]
4                              [factsguid, societi, motiv]
5        [huge, fan, fare, big, talk, leav, chao, pay, ...
6                                  [camp, tomorrow, danni]
7        [next, school, year, year, exam, think, school...
8        [love, land, allin, cav, champion, cleveland, ...
9                                             [welcom, gr]
10       [ireland, consum, price, index, mom, climb, pr...
11       [selfish, orlando, standwithorlando, pulseshoo...
12                 [get, see, daddi, today, day, gettingf]
13       [cnn, call, michigan, middl, school, build, wa...
14       [comment, australia, opkillingbay, seashepherd...
15        [ouch, junior, angri, got, junior, yugyoem, omg]
16                            [thank, paner, thank, posi

In [42]:
combine_df['tweet_lemmatized']

0        [father, dysfunctional, selfish, drag, kid, dy...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                            [model, love, take, time, ur]
4                        [factsguide, society, motivation]
5        [huge, fan, fare, big, talk, leave, chaos, pay...
6                                  [camp, tomorrow, danny]
7        [next, school, year, year, exam, think, school...
8        [love, land, allin, cavs, champion, cleveland,...
9                                            [welcome, gr]
10       [ireland, consumer, price, index, mom, climbed...
11       [selfish, orlando, standwithorlando, pulseshoo...
12               [get, see, daddy, today, day, gettingfed]
13       [cnn, call, michigan, middle, school, build, w...
14       [comment, australia, opkillingbay, seashepherd...
15        [ouch, junior, angry, get, junior, yugyoem, omg]
16                   [thankful, paner, thankful, positiv

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
●	Исключим стоп-слова с помощью stop_words='english'.
●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


In [43]:
documents = combine_df['tweet_stemmed'].apply(lambda x: " ".join(x)) 

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(documents)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words[:10].toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
documents = combine_df['tweet_lemmatized'].apply(lambda x: " ".join(x)) 

In [46]:
# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(documents)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words[:10].toarray(), columns = feature_names)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yes,yesterday,yo,yoga,york,young,youth,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
import pickle
result = combine_df
dee = open('combine_df.dat','ab')
pickle.dump(result,dee)
dee.close()

In [48]:
#dee = open('tweet_lemmatized.dat','rb')
#result = pickle.load(dee)
#dee.close()
#result

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
●	Исключим стоп-слова с помощью stop_words='english'.
●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [49]:
documents = combine_df['tweet_stemmed'].apply(lambda x: " ".join(x)) 

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')
values = tfidf_vectorizer.fit_transform(documents)

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values[:10].toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.403633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
documents = combine_df['tweet_lemmatized'].apply(lambda x: " ".join(x)) 

In [52]:
values = tfidf_vectorizer.fit_transform(documents)

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values[:10].toarray(), columns = feature_names)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yes,yesterday,yo,yoga,york,young,youth,youtube,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


3. Натренируем gensim.models.Word2Vec модель на наших данных.
●	Тренировать будем на токенизированных твитах combine_df['tweet_token']
●	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
●	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


In [53]:
!pip install gensim

In [54]:
from gensim.models import Word2Vec
modelW2V = Word2Vec(sentences=combine_df['tweet_lemmatized'], size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [55]:
l = len(combine_df['tweet_lemmatized'])
modelW2V.train([["hello", "world"]], total_examples=l, epochs=20)

(40, 40)

In [56]:
list(modelW2V.wv.vocab.keys())[:10]

['father',
 'dysfunctional',
 'selfish',
 'drag',
 'kid',
 'dysfunction',
 'run',
 'thanks',
 'lyft',
 'credit']

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [57]:
# выведим слово наиболее близкое к 'father'
result = modelW2V.most_similar(positive=['father'])
print("{}: {:.4f}".format(*result[0]))

dad: 0.9257


In [58]:
# выведим слово наиболее близкое к '"trump'
result = modelW2V.most_similar(positive=['trump'])
print("{}: {:.4f}".format(*result[0]))

republican: 0.8775


Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [59]:
modelW2V['food']

array([ 2.68488646e-01,  2.10635930e-01,  1.26947626e-01,  1.48740103e-02,
       -1.82228297e-01, -1.78644191e-02,  1.28989860e-01, -1.20919131e-01,
        1.88916981e-01, -1.69360653e-01,  6.60377741e-02,  2.63126224e-01,
       -6.10124618e-02,  1.97807342e-01,  5.39278202e-02, -5.65540418e-02,
        1.44487485e-01, -1.63901493e-01, -1.72741279e-01,  3.22252244e-01,
        2.01328442e-01, -3.61459881e-01, -1.91811398e-01,  1.71076164e-01,
       -1.02036856e-01,  4.73915748e-02,  1.80830866e-01, -1.54777363e-01,
       -1.15201838e-01, -1.72773749e-02, -2.41463840e-01, -2.05460805e-02,
       -2.23328024e-01, -2.39669606e-01,  2.95488805e-01,  1.06317438e-02,
        2.99348384e-01, -3.92366767e-01,  2.66671836e-01, -3.84514809e-01,
       -1.91563070e-01,  5.04891984e-02,  3.19561623e-02, -2.07883671e-01,
       -1.04230689e-02,  3.85799110e-01, -2.52776872e-02, -1.42869711e-01,
       -4.37801242e-01, -2.01717764e-02,  1.30325211e-02,  3.55652183e-01,
        3.91330987e-01, -

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).


In [60]:
def get_vector(tweet, model, size):
    vector = np.zeros(size)
    norm = 0
    for word in tweet:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    return vector

In [61]:
combine_df['modelW2V_vector'] = combine_df['tweet_token'].apply(lambda x: get_vector(x, modelW2V, 200))

In [62]:
tweets = list(combine_df['tweet_token'])

In [63]:
wordvec = []
for tweet in tweets:
    wordvec.append(get_vector(tweet, modelW2V, 200))
wordvec_df = np.array(wordvec)
wordvec_df.shape

(49159, 200)

In [64]:
combine_df['modelW2V_vector']

0        [-0.0805285211120333, 0.05396790852371071, 0.2...
1        [-0.12390832078050483, 0.034736652223562654, 0...
2        [-0.2952888458967209, -0.08504673466086388, 0....
3        [-0.06578852950284879, -0.05843752451861898, 0...
4        [-0.0520211768647035, 0.11725588887929916, 0.3...
5        [-0.1329496776064237, 0.08391263620514008, 0.2...
6        [-0.1339208446443081, 0.08242642879486084, 0.2...
7        [-0.09116719961166382, -0.024419031478464602, ...
8        [-0.08626534640789033, -0.046913688676431774, ...
9        [-0.09613505564630032, 0.030341148318257183, 0...
10       [-0.26295594584483367, 0.08589827097379245, 0....
11       [-0.0613036033593946, -0.024352875020768907, 0...
12       [-0.17221662998199463, 0.11937511307187379, 0....
13       [-0.11052707396447659, 0.013661057426361367, 0...
14       [-0.1454604615767797, -0.02942038710332579, 0....
15       [-0.14037332832813262, -0.06636816635727882, 0...
16       [-0.12906241975724697, 0.12508995458483696, 0..